# Buy and Hold Strategy using MA120

## Background



To-Do:


---
## 0. Imports

In [1]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.preprocessing import StandardScaler

import sys
sys.path.append('../../functions_library')

from functions import ROC, model_selection, Backtesting

#import sklearn 
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
import sklearn.ensemble

# Classification Metrics 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc #plot_roc_curve, auc
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

# ML models 
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import xgboost as xgb

# libraries for Shapely analysis
import shap 

# Visualization libraries
import matplotlib.pyplot as plt
import hvplot.pandas

seed = 42

---
## 1. Data Loading

Read the CSV file into Pandas DataFrame

In [2]:
# Import the OHLCV dataset into a Pandas Dataframe
trading_df = pd.read_csv(
    Path("../../Resources/ETHUSDT-1h-data.csv"), 
    index_col="timestamp", 
    infer_datetime_format=True, 
    parse_dates=True
)

trading_df = trading_df.drop(["close_time","quote_av","trades","tb_base_av","tb_quote_av","ignore"], axis =1)

# Review the DataFrame
trading_df.head()

,open,high,low,close,volume
timestamp,,,,,
2017-08-17 04:00:00,301.13,302.57,298.0,301.61,125.66877
2017-08-17 05:00:00,301.61,303.28,300.0,303.10,377.67246
2017-08-17 06:00:00,302.40,304.44,301.9,302.68,303.86672
2017-08-17 07:00:00,302.68,307.96,302.6,307.96,754.74510
2017-08-17 08:00:00,307.95,309.97,307.0,308.62,150.75029


In [3]:
import yfinance as yf

# Download SPY data
spy = yf.download('SPY', start='2017-01-01', end='2023-03-10')

# Print the first few rows of the data
spy


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-03,225.039993,225.830002,223.880005,225.240005,202.856506,91366500
2017-01-04,225.619995,226.750000,225.610001,226.580002,204.063339,78744400
2017-01-05,226.270004,226.580002,225.479996,226.399994,203.901276,78379000
2017-01-06,226.529999,227.750000,225.899994,227.210007,204.630722,71559900
2017-01-09,226.910004,227.070007,226.419998,226.460007,203.955292,46939700
...,...,...,...,...,...,...
2023-03-03,399.709991,404.450012,399.029999,404.190002,404.190002,90038300
2023-03-06,405.049988,407.450012,404.010010,404.470001,404.470001,72795900
2023-03-07,404.420013,404.670013,397.630005,398.269989,398.269989,108310600


In [4]:
import finta as ft
from finta import TA
import talib

feat_df = trading_df.copy()

# List of time periods to use for Moving Averages calculation
timeperiods = [5,7,14,20,30,50,70,100,150,200,2400]



# Calculate SMAs and add them to the DataFrame
for t in timeperiods:
    #tsma = TA.SMA(df, t).shift(1)
    sma = TA.SMA(feat_df, t)
    feat_df[f'sma_{t}'] = sma

    

In [5]:
# rename column 'A' to 'new_name'
feat_df = feat_df.rename(columns={'close': 'eth_close'})
feat_df = feat_df.drop(["open","high","low","volume"], axis =1)


feat_df = feat_df.dropna()
display(feat_df)

,eth_close,sma_5,sma_7,sma_14,sma_20,sma_30,sma_50,sma_70,sma_100,sma_150,sma_200,sma_2400
timestamp,,,,,,,,,,,,
2017-11-25 09:00:00,452.46,462.814,462.322857,463.576429,459.5045,444.649667,429.5126,413.350571,399.2934,386.030467,372.81055,311.128733
2017-11-25 10:00:00,460.00,461.418,462.681429,463.577143,460.5705,446.383333,431.0104,414.707429,400.3066,386.780600,373.44975,311.194729
2017-11-25 11:00:00,464.86,460.122,462.704286,464.156429,461.0440,448.082000,432.4718,416.159143,401.3224,387.559667,374.11470,311.262129
2017-11-25 12:00:00,460.86,459.488,461.830000,464.290000,461.5795,449.861000,433.7358,417.557000,402.2714,388.248467,374.76815,311.328037
2017-11-25 13:00:00,468.05,461.246,461.360000,464.222143,462.3855,451.862667,435.1362,419.026429,403.2978,388.955533,375.45345,311.394742
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-23 18:00:00,1628.85,1619.534,1622.211429,1628.433571,1630.2090,1631.986000,1633.1332,1638.140286,1614.2020,1594.658733,1585.73290,1325.842521
2023-01-23 19:00:00,1619.88,1623.684,1620.017143,1627.553571,1629.8310,1631.581667,1632.4240,1637.898143,1615.0699,1594.989400,1586.16555,1325.983108
2023-01-23 20:00:00,1634.80,1626.474,1621.764286,1627.383571,1629.7840,1631.324667,1631.8958,1637.653286,1616.0668,1595.348533,1586.67980,1326.130562


In [6]:
# Load the ETHUSD OHLCV data
resampled_df = feat_df.copy()

# resample to 24H frequency and calculate OHLCV
resampled_df = feat_df.resample('24H').agg({'eth_close': 'last', 'sma_2400':'last'})

# add spy data
resampled_df['spy_close'] = spy['Adj Close']

# Interpolate missing values in the 'spy_close' column using linear interpolation
resampled_df['spy_close'] = resampled_df['spy_close'].interpolate(method='linear')

---
# 2. MODELS
# 2.1 Dolar-Cost averaging with SMA100 comparison

This code implements a dollar cost averaging strategy for investing in two assets: SPY and ETH. It resamples the ETHUSD data to 24H frequency and calculates the OHLCV, then adds SPY data and interpolates missing values in the 'spy_close' column using linear interpolation. It filters for the time range of interest and initializes columns. Then, it calculates the total amount of ETH and SPY that can be bought with a daily contribution, taking into account a 1% fee. It also implements the SMA strategy by buying ETH when its price is below the 100-day simple moving average. The code calculates the amount of USD invested and the cumulative amount of ETH bought and invested in SPY over time, as well as the value of the holdings in USD. Finally, it plots the cumulative ETH holdings value, ETH holdings value using SMA strategy, and SPY investment value.

### variables: 

- **daily_contribution** = amount of USD to be invested each day 
- **eth_daily_add** = amount of ETH that can be added daily, by using daily_contribution to buy ETH
- **spy_daily_add** = amount of SPY that can be added daily, by using daily_contribution to buy SPY

### columns: 
- spy_close = price of SPYUSD
- eth_close = price of ETHUSD
- eth_bought = amount of ETH bought daily 
- spy_bought = amount of SPY bought daily 
- usd_invested = total aount of USD invested 

### DCA+SMA - Try 1

In [7]:
import pandas as pd
import numpy as np

# Load the ETHUSD OHLCV data
portfolio_df = resampled_df.copy()


# Filter for the time range of interest
start_date = '2017-1-18'
end_date = '2023-01-22'
portfolio_df = portfolio_df.loc[start_date:end_date]
daily_contribution = 500

# initialize columns
portfolio_df['eth_bought'] = portfolio_df['spy_bought'] =0
portfolio_df['spy_investment'] = portfolio_df['usd_invested'] = 0

# Calculate the total amount of ETH and SPY that can be bought with the daily contribution - inc 1% fee
portfolio_df['eth_daily_add'] = (daily_contribution / portfolio_df['eth_close']) * (1 - 0.01)
portfolio_df['spy_daily_add'] = (daily_contribution / portfolio_df['spy_close']) * (1 - 0.01)


# Calculate the amount of ETH that can be bought and the amount to be invested in SPY
for i, row in portfolio_df.iterrows():
    eth_daily_add = portfolio_df.at[i, 'eth_daily_add']
    spy_daily_add = portfolio_df.at[i, 'spy_daily_add']
    
    portfolio_df.at[i, 'eth_bought'] = eth_daily_add
    portfolio_df.at[i, 'spy_bought'] = spy_daily_add
    
    # SMA strategy
    if portfolio_df.at[i, 'eth_close'] < portfolio_df.at[i, 'sma_2400']: 
        portfolio_df.at[i, 'eth_sma_bought'] = eth_daily_add
        portfolio_df.at[i, 'spy_sma_bought'] = 0
    else:
        portfolio_df.at[i, 'eth_sma_bought'] = 0
        portfolio_df.at[i, 'spy_sma_bought'] = spy_daily_add
        
# Calculate the amount of USD invested       
portfolio_df['usd_invested'] = daily_contribution
portfolio_df['usd_invested'] = portfolio_df['usd_invested'].cumsum()
    
# Calculate the cumulative amount of ETH bought and invested in SPY over time
portfolio_df['eth_cumulative'] = portfolio_df['eth_daily_add'].cumsum()
portfolio_df['spy_cumulative'] = portfolio_df['spy_daily_add'].cumsum()
portfolio_df['eth_sma_cumulative'] = portfolio_df['eth_sma_bought'].cumsum()
portfolio_df['spy_sma_cumulative'] = portfolio_df['spy_sma_bought'].cumsum()


# Calculate the value of the ETH holdings and SPY investment in USD
portfolio_df['eth_usd_value'] = portfolio_df['eth_cumulative'] * portfolio_df['eth_close']
portfolio_df['spy_usd_value'] = portfolio_df['spy_cumulative'] * portfolio_df['spy_close']
portfolio_df['eth_sma_usd_value'] = portfolio_df['eth_sma_cumulative'] * portfolio_df['eth_close']
portfolio_df['spy_sma_usd_value'] = portfolio_df['spy_sma_cumulative'] * portfolio_df['spy_close']
portfolio_df['strategy_sma_usd_value'] = portfolio_df['eth_sma_usd_value'] + portfolio_df['spy_sma_usd_value']
#portfolio_df['eth_average_price'] = portfolio_df['eth_sma_bought'] / portfolio_df['usd_invested']



# Plot the cumulative ETH holdings value, ETH holdings value using SMA strategy, and SPY investment value
portfolio_df[['eth_usd_value','spy_usd_value','strategy_sma_usd_value','usd_invested']].hvplot( title='Portfolio Performance')


:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [8]:
#Plot the cumulative ETH holdings value, ETH holdings value using SMA strategy, and SPY investment value
portfolio_df[['spy_usd_value','usd_invested']].hvplot( title='SPY Performance')


:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [9]:
#Plot the cumulative ETH holdings value, ETH holdings value using SMA strategy, and SPY investment value
#portfolio_df[['eth_average_price','eth_close']].hvplot( title='ETH average price Performance')

In [10]:
portfolio_df[['eth_usd_value','spy_usd_value','strategy_sma_usd_value','usd_invested']]

,eth_usd_value,spy_usd_value,strategy_sma_usd_value,usd_invested
timestamp,,,,
2017-11-25,4.950000e+02,NaN,NaN,500
2017-11-26,9.910933e+02,NaN,NaN,1000
2017-11-27,1.506372e+03,4.950000e+02,4.950000e+02,1500
2017-11-28,1.981789e+03,9.950217e+02,9.950217e+02,2000
2017-11-29,2.287737e+03,1.489416e+03,1.489416e+03,2500
...,...,...,...,...
2023-01-18,3.953739e+06,1.153698e+06,2.868273e+06,940500
2023-01-19,4.057981e+06,1.145794e+06,2.924655e+06,941000
2023-01-20,4.339535e+06,1.167634e+06,3.098634e+06,941500


### DCA+SMA - Try 2

- [x] Coherent results
- [ ] Check no using data from the future 

In [11]:
import pandas as pd
import numpy as np

# Load the ETHUSD OHLCV data
portfolio_df = resampled_df.copy()

# resample to 24H frequency and calculate OHLCV
portfolio_df = portfolio_df.resample('24H').agg({'eth_close': 'last', 'sma_2400':'last'})

# add spy data
portfolio_df['spy_close'] = spy['Adj Close']

# Interpolate missing values in the 'spy_close' column using linear interpolation
portfolio_df['spy_close'] = portfolio_df['spy_close'].interpolate(method='linear')


# Filter for the time range of interest
start_date = '2022-1-18'
end_date = '2023-01-22'
portfolio_df = portfolio_df.loc[start_date:end_date]
daily_contribution = 500

# initialize columns
portfolio_df['eth_bought'] = portfolio_df['spy_bought'] =0
portfolio_df['spy_investment'] = portfolio_df['usd_invested'] = 0

# Calculate the total amount of ETH and SPY that can be bought with the daily contribution - inc 1% fee
portfolio_df['eth_daily_add'] = (daily_contribution / portfolio_df['eth_close']) * (1 - 0.01)
portfolio_df['spy_daily_add'] = (daily_contribution / portfolio_df['spy_close']) * (1 - 0.01)

# Calculate the amount of ETH that can be bought and the amount to be invested in SPY
for i, row in portfolio_df.iterrows():
    eth_daily_add = portfolio_df.at[i, 'eth_daily_add']
    spy_daily_add = portfolio_df.at[i, 'spy_daily_add']
    
    portfolio_df.at[i, 'eth_bought'] = eth_daily_add
    portfolio_df.at[i, 'spy_bought'] = spy_daily_add
            
# Calculate the amount of USD invested       
portfolio_df['usd_invested'] = daily_contribution
portfolio_df['usd_invested'] = portfolio_df['usd_invested'].cumsum()
    
# Calculate the cumulative amount of ETH bought and invested in SPY over time
portfolio_df['eth_cumulative'] = portfolio_df['eth_daily_add'].cumsum()
portfolio_df['spy_cumulative'] = portfolio_df['spy_daily_add'].cumsum()

# Calculate the value of the ETH holdings and SPY investment in USD
portfolio_df['eth_usd_value'] = portfolio_df['eth_cumulative'] * portfolio_df['eth_close']
portfolio_df['spy_usd_value'] = portfolio_df['spy_cumulative'] * portfolio_df['spy_close']

# # SMA strategy
for i, row in portfolio_df.iterrows():
    
    if portfolio_df.at[i, 'eth_close'] < portfolio_df.at[i, 'sma_2400']: 
        portfolio_df.at[i, 'eth_sma_bought'] = eth_daily_add
        portfolio_df.at[i, 'spy_sma_bought'] = 0
        portfolio_df.at[i, 'usd_invested_in_eth'] = daily_contribution
        portfolio_df.at[i, 'eth_average_price'] = portfolio_df.at[i, 'usd_invested_in_eth'] / portfolio_df.at[i, 'eth_sma_bought']
    else:
        portfolio_df.at[i, 'eth_sma_bought'] = 0
        portfolio_df.at[i, 'spy_sma_bought'] = spy_daily_add
        
portfolio_df['eth_sma_cumulative'] = portfolio_df['eth_sma_bought'].cumsum()
portfolio_df['spy_sma_cumulative'] = portfolio_df['spy_sma_bought'].cumsum()

portfolio_df['eth_sma_usd_value'] = portfolio_df['eth_sma_cumulative'] * portfolio_df['eth_close']
portfolio_df['spy_sma_usd_value'] = portfolio_df['spy_sma_cumulative'] * portfolio_df['spy_close']
portfolio_df['strategy_sma_usd_value'] = portfolio_df['eth_sma_usd_value'] + portfolio_df['spy_sma_usd_value']

        
# Plot the cumulative ETH holdings value, ETH holdings value using SMA strategy, and SPY investment value
portfolio_df[['eth_usd_value','spy_usd_value','strategy_sma_usd_value','usd_invested']].hvplot( title='Portfolio Performance')


:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

---
## 2.2 DCA+SMA+TP

Adding a Take Profit condition to sell ETH when the the average purchased price has gone up by 30%. 

### 2.2.3 DCA+SMA+TP - Try 3

- [ ] fix portfolio value, sell option doesn't work properly


In [12]:
portfolio_df.columns

Index(['eth_close', 'sma_2400', 'spy_close', 'eth_bought', 'spy_bought',
       'spy_investment', 'usd_invested', 'eth_daily_add', 'spy_daily_add',
       'eth_cumulative', 'spy_cumulative', 'eth_usd_value', 'spy_usd_value',
       'eth_sma_bought', 'spy_sma_bought', 'usd_invested_in_eth',
       'eth_average_price', 'eth_sma_cumulative', 'spy_sma_cumulative',
       'eth_sma_usd_value', 'spy_sma_usd_value', 'strategy_sma_usd_value'],
      dtype='object')

--- 
Try 3

In [13]:

# Load the time series data
df =  resampled_df[['eth_close', 'sma_2400', 'spy_close']].copy()

# Filter for the time range of interest
start_date = '2020-1-18'
end_date = '2023-01-22'
df = df.loc[start_date:end_date]

# Initialize the strategy variables
capital = capital_sma = 100000        # inital capital
daily_contribution_low = 100
                   # number of days used for dollar-cost averagaging
fee = 0.01                            # trading exchange fees. Ex 1% =  0.01 
eth_owned = eth_bought = 0
eth_cost_basis = 0
eth_value = eth_value_sma = eth_usd_value = eth_own = spy_usd_value = 0
eth_sell_price = 0
eth_cumulative = spy_cumulative = 0
usd_invested = usd_invested_in_sma = earned_usd = 0

########## PARAMETERS TO OPTIMIZE THE MODEL ##########
dca_duration = 45  # dollar cost averaging timeframe to use the capital
sell_threshold = 3 # if 2 --> sell when ETH price is 2x cost basis
sma_coefficient = 3  # if 1 --> accumulate when ETH price is < 1 x sma_2400
######################################################

# calulate daily contribution for both dollar cost averaging 
daily_contribution = capital / dca_duration
daily_contribution_sma = capital_sma / dca_duration

buy_below_sma = True
backtest_results = []

# making sure we are not using data from the future 
df['eth_close_prev_day'] = df['eth_close'].shift(-1)
df['sma_2400_prev_day'] = df['sma_2400'].shift(-1)
df['spy_close_prev_day'] = df['spy_close'].shift(-1)

df = df.dropna()

# Loop through each day in the dataset
for date, row in df.iterrows():
    eth_close_prev_day = row['eth_close_prev_day']
    sma_2400_prev_day = row['sma_2400_prev_day']
    eth_close = row['eth_close']
    spy_close = row['spy_close']
    sma_2400 = row['sma_2400']
    
    #BUY
    # Buy ETH with daily contribution if price is below SMA
    if eth_close_prev_day < sma_coefficient*sma_2400_prev_day or buy_below_sma:
        
        # check there is still capital available for buying
        if capital_sma > 0: 
            capital_sma -= daily_contribution_sma
            if earned_usd >0 :  earned_usd -= daily_contribution_sma

        else:
            capital_sma =0
            daily_contribution_sma = daily_contribution_low

        usd_invested_in_sma += daily_contribution_sma
        eth_bought = daily_contribution_sma / eth_close
        eth_owned += eth_bought
        eth_cost_basis = (eth_cost_basis * eth_value + daily_contribution_sma) / (eth_value + eth_bought)
        backtest_results.append(('buy', date, eth_close))
            
    # SELL    
    # Sell all ETH if price is above sell threshold
    if eth_close_prev_day > sell_threshold * eth_cost_basis and eth_owned > 0:
        #eth_sell_price = eth_close
        capital_sma += eth_value
        earned_usd = capital_sma
        eth_owned = 0
        eth_cost_basis = 0
        #eth_value = 0
        daily_contribution_sma = capital_sma / dca_duration
        buy_below_sma = False
        backtest_results.append(('sell', date, eth_close))
    
    # Restart dollar-cost averaging if price goes below SMA
    if eth_close < sma_coefficient*sma_2400 : #and not buy_below_sma:
        buy_below_sma = True
    
    eth_value = eth_owned * eth_close
    
    # Calculate the portfolio value
    portfolio_value = earned_usd + eth_value
    
    # Store the portfolio value and daily contribution for this day
    backtest_results.append(('hold', date, eth_close, portfolio_value, daily_contribution_sma, capital_sma,usd_invested_in_sma))
    
    # Compare with dollar-cost averaging for SPY and ETH
    # check first if capital available to invest
    if (capital-usd_invested) <= 0 : 
        usd_invested = capital 
        daily_contribution =0
    else : usd_invested += daily_contribution
    
    # Calculate the total amount of ETH and SPY that can be bought with the daily contribution - inc 1% fee
    eth_daily_add = daily_contribution / eth_close * (1 - fee)
    spy_daily_add = daily_contribution / spy_close * (1 - fee)
    eth_cumulative += eth_daily_add
    spy_cumulative += spy_daily_add

    eth_usd_value =  eth_cumulative * eth_close 
    spy_usd_value =  spy_cumulative * spy_close  
    
    # Store the portfolio value and daily contribution for this day
    backtest_results.append(('hold', date, eth_close, portfolio_value, daily_contribution_sma,capital_sma, eth_usd_value,spy_usd_value,usd_invested_in_sma))
    
# Convert the backtest results to a DataFrame
backtest_df = pd.DataFrame(backtest_results, columns=['action', 'date', 'eth_close', 'portfolio_value', 'daily_contribution_sma', 'capital_sma','eth_usd_value','spy_usd_value','usd_invested_in_sma'])
backtest_df.set_index('date', inplace=True)
backtest_df = backtest_df.dropna()

# interpolate missing values
backtest_df = backtest_df.interpolate()

# Plot the cumulative ETH holdings value, ETH holdings value using SMA strategy, and SPY investment value
backtest_df[['portfolio_value','usd_invested_in_sma','eth_usd_value','spy_usd_value']].hvplot.line(title='Portfolio Performance',width=1200, height=400)

#backtest_df[['eth_close','usd_invested']].hvplot.line(title='Portfolio Performance',width=1200, height=400)

'''
fig, ax = plt.subplots(2, 1, figsize=(12, 8))
backtest_df['portfolio_value'].plot(ax=ax[0], label='Portfolio Value',kind='line')
backtest_df['usd_invested_'].plot(ax=ax[0], label='usd_invested')
backtest_df['eth_usd_value'].plot(ax=ax[0], label='eth_usd_value')
backtest_df['spy_usd_value'].plot(ax=ax[0], label='spy_usd_value')
ax[0].legend()
ax[0].set_title('Portfolio Value and Daily Contribution')
backtest_df['eth_close'].plot(ax=ax[1], label='ETH Close')
ax[1].legend()
ax[1].set_title('ETH Close')
plt.show()
'''


"\nfig, ax = plt.subplots(2, 1, figsize=(12, 8))\nbacktest_df['portfolio_value'].plot(ax=ax[0], label='Portfolio Value',kind='line')\nbacktest_df['usd_invested_'].plot(ax=ax[0], label='usd_invested')\nbacktest_df['eth_usd_value'].plot(ax=ax[0], label='eth_usd_value')\nbacktest_df['spy_usd_value'].plot(ax=ax[0], label='spy_usd_value')\nax[0].legend()\nax[0].set_title('Portfolio Value and Daily Contribution')\nbacktest_df['eth_close'].plot(ax=ax[1], label='ETH Close')\nax[1].legend()\nax[1].set_title('ETH Close')\nplt.show()\n"

In [14]:
# Plot the cumulative ETH holdings value, ETH holdings value using SMA strategy, and SPY investment value
backtest_df[['portfolio_value','usd_invested_in_sma','capital_sma','eth_usd_value','spy_usd_value']].hvplot.line(title='Portfolio Performance',width=1200, height=400)

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [15]:
# Plot the cumulative ETH holdings value, ETH holdings value using SMA strategy, and SPY investment value
backtest_df[['eth_close','daily_contribution_sma']].hvplot.line(title='Portfolio Performance',width=1200, height=400)

:NdOverlay   [Variable]
   :Curve   [date]   (value)